In [1]:
import pandas as pd
import numpy as np
import nltk
import re
sent_token = nltk.sent_tokenize
import csv  
from nltk import sent_tokenize, word_tokenize, pos_tag
import re
from sklearn.feature_extraction.text import CountVectorizer
wpt = nltk.WordPunctTokenizer()


file_path_comments = r'~/Documents/Springboard/Springboard/Data/cleaned_comment_data.csv'

#file_path_comments = r'/mnt/c/Users/msteele9/Documents/Springboard/Springboard/Data/cleaned_comment_data.csv'
clean_comments = pd.read_csv(file_path_comments, index_col = False)

In [2]:
from sklearn.model_selection import train_test_split
import random
from datetime import datetime

X = clean_comments['commentBody']
y = clean_comments['recommendations']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=random.seed(datetime.now()))

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
 
vectorizer = CountVectorizer(binary=False, stop_words=stopwords.words('english'), 
                             lowercase=True, min_df=1, max_df=0.9, max_features=5000)
X_train_onehot = vectorizer.fit_transform(X_train)

In [4]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
 
    model.add(Dense(units=500, activation='tanh', input_dim=len(vectorizer.get_feature_names())))

    model.add(Dense(units=400, activation='relu'))

    model.add(Dense(units=1, activation='relu'))
 
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

seed = 7
np.random.seed(seed)
# load dataset

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_onehot, y_train)

Using TensorFlow backend.
C:\Users\msteele9\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\msteele9\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\msteele9\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [5]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.223200 using {'batch_size': 100, 'epochs': 50}
0.135125 (0.030969) with: {'batch_size': 10, 'epochs': 10}
0.129575 (0.003686) with: {'batch_size': 10, 'epochs': 50}
0.115975 (0.007959) with: {'batch_size': 10, 'epochs': 100}
0.164200 (0.005115) with: {'batch_size': 20, 'epochs': 10}
0.151850 (0.004913) with: {'batch_size': 20, 'epochs': 50}
0.139100 (0.007326) with: {'batch_size': 20, 'epochs': 100}
0.176700 (0.027827) with: {'batch_size': 40, 'epochs': 10}
0.176800 (0.008613) with: {'batch_size': 40, 'epochs': 50}
0.165225 (0.007088) with: {'batch_size': 40, 'epochs': 100}
0.216875 (0.004961) with: {'batch_size': 60, 'epochs': 10}
0.198200 (0.005164) with: {'batch_size': 60, 'epochs': 50}
0.186625 (0.007020) with: {'batch_size': 60, 'epochs': 100}
0.222425 (0.005550) with: {'batch_size': 80, 'epochs': 10}
0.220450 (0.004945) with: {'batch_size': 80, 'epochs': 50}
0.196425 (0.003424) with: {'batch_size': 80, 'epochs': 100}
0.221125 (0.004094) with: {'batch_size': 100, 'epochs':

In [6]:
batch_size = [10, 20, 40, 60, 80, 100, 150, 200, 250, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000]
epochs = [10]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train_onehot, y_train)

In [7]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.224425 using {'batch_size': 100, 'epochs': 10}
0.127000 (0.014607) with: {'batch_size': 10, 'epochs': 10}
0.147725 (0.021218) with: {'batch_size': 20, 'epochs': 10}
0.200725 (0.018156) with: {'batch_size': 40, 'epochs': 10}
0.209175 (0.009844) with: {'batch_size': 60, 'epochs': 10}
0.218400 (0.000219) with: {'batch_size': 80, 'epochs': 10}
0.224425 (0.007583) with: {'batch_size': 100, 'epochs': 10}
0.215850 (0.010240) with: {'batch_size': 150, 'epochs': 10}
0.215800 (0.005768) with: {'batch_size': 200, 'epochs': 10}
0.220325 (0.006775) with: {'batch_size': 250, 'epochs': 10}
0.224275 (0.002925) with: {'batch_size': 300, 'epochs': 10}
0.195625 (0.012638) with: {'batch_size': 400, 'epochs': 10}
0.182475 (0.012463) with: {'batch_size': 500, 'epochs': 10}
0.165750 (0.010133) with: {'batch_size': 600, 'epochs': 10}
0.145975 (0.013723) with: {'batch_size': 700, 'epochs': 10}
0.127475 (0.011878) with: {'batch_size': 800, 'epochs': 10}
0.125925 (0.012141) with: {'batch_size': 900, 'epo